# EDA de **Telefonía Fija: Accesos Totales**

Cuaderno para **Visual Studio/VS Code** (Python) con pasos documentados en Markdown y exportación del CSV 'limpio'.

**Columnas esperadas:** `Registro_ID, Año, Trimestre, Hogares, Comercial, Gobierno, Otros, Total, TiempoTrim_ID`.

## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (11, 5),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
> **Ruta proporcionada:**

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Accesos Totales_con_ID.csv`

Usa `encoding='utf-8-sig'` para preservar acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Accesos Totales_con_ID.csv'

df = pd.read_csv(ruta, encoding='utf-8-sig')
df_raw = df.copy()

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial
- Tipos de datos (`df.info()`)
- Nulos y duplicados
- Rangos básicos por métricas principales

In [ ]:
df.info()

print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

cols_rango = ['Hogares','Comercial','Gobierno','Otros','Total']
for col in cols_rango:
    if col in df.columns:
        print(col, 'min=', df[col].min(), 'max=', df[col].max(), 'mean=', float(df[col].mean()))

## 4. Validaciones de consistencia
- Conversión de columnas numéricas
- Chequeo de valores negativos
- Verificación **Total vs suma de componentes (Hogares+Comercial+Gobierno+Otros)**
- Construcción de `Periodo` (YYYY-Qn)

In [ ]:
num_cols = ['Registro_ID','Año','Trimestre','Hogares','Comercial','Gobierno','Otros','Total','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())

# Suma de componentes
if set(['Hogares','Comercial','Gobierno','Otros']).issubset(df.columns):
    comp_sum = df[['Hogares','Comercial','Gobierno','Otros']].sum(axis=1)
    consist = (df['Total'] == comp_sum) if 'Total' in df.columns else (comp_sum.notna())
    print('Consistentes Total == suma:', consist.mean(), '(', int(consist.sum()), 'de', len(consist), ')')
    df['dif_total'] = df.get('Total', pd.Series(index=df.index)) - comp_sum

# Periodo
df['Periodo'] = pd.PeriodIndex.from_fields(year=df['Año'], quarter=df['Trimestre']).astype(str)

## 5. Limpieza y ordenamiento
- Eliminar duplicados
- Orden por `Año`, `Trimestre`
- Columnas auxiliares conservadas para auditoría (`dif_total`)

In [ ]:
df = df.drop_duplicates().sort_values(['Año','Trimestre']).reset_index(drop=True)
display(df.head())

## 6. Variables derivadas
- Porcentajes de composición sobre `Total`
- Tendencia reciente de participación de segmentos

In [ ]:
total_nonzero = df['Total'].replace(0, pd.NA)
df['porc_hogares'] = (df['Hogares'] / total_nonzero) * 100
df['porc_comercial'] = (df['Comercial'] / total_nonzero) * 100
df['porc_gobierno'] = (df['Gobierno'] / total_nonzero) * 100
df['porc_otros'] = (df['Otros'] / total_nonzero) * 100

display(df[['Año','Trimestre','porc_hogares','porc_comercial','porc_gobierno','porc_otros']].tail(8))

## 7. Visualizaciones
- Serie temporal de **Total**
- Participación (%) de segmentos (últimos 12 trimestres)

In [ ]:
import matplotlib.ticker as mtick

# 7.1 Tendencia del Total
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Total', marker='o', color='#2E86C1')
plt.xticks(rotation=45, ha='right')
plt.title('Accesos totales de telefonía fija por trimestre')
plt.xlabel('Periodo'); plt.ylabel('Total'); plt.tight_layout(); plt.show()

# 7.2 Participación por segmentos (últimos 12 trimestres)
tail = df.tail(12)
plt.figure()
sns.lineplot(data=tail, x='Periodo', y='porc_hogares', marker='o', label='Hogares', color='#1ABC9C')
sns.lineplot(data=tail, x='Periodo', y='porc_comercial', marker='o', label='Comercial', color='#E67E22')
sns.lineplot(data=tail, x='Periodo', y='porc_gobierno', marker='o', label='Gobierno', color='#8E44AD')
sns.lineplot(data=tail, x='Periodo', y='porc_otros', marker='o', label='Otros', color='#34495E')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.xticks(rotation=45, ha='right')
plt.title('Participación (%) por segmento (últimos 12 trimestres)')
plt.xlabel('Periodo'); plt.ylabel('% del total'); plt.legend(); plt.tight_layout(); plt.show()

## 8. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)